In [ ]:
# ============================================================================
# CARGA DE ARCHIVO CON ÁREA DE INTERÉS
# ============================================================================

import geopandas as gpd
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

if IN_COLAB:
    from google.colab import files
    
    print("="*70)
    print("📁 SUBIR ARCHIVO DE ÁREA DE INTERÉS")
    print("="*70)
    print("\n📋 Formatos aceptados:")
    print("   • GeoPackage (.gpkg) - Recomendado")
    print("   • Shapefile (.shp + archivos auxiliares)")
    print("   • GeoJSON (.geojson)")
    print("\n👇 Haz click en 'Elegir archivos' y selecciona tu archivo(s)")
    print("="*70)
    
    uploaded = files.upload()
    filename = list(uploaded.keys())[0]
    
    print(f"\n✅ Archivo recibido: {filename}")
else:
    # Modo local: usar archivo del proyecto
    filename = 'data/municipios_seleccionados.gpkg'
    print(f"💻 Modo local: usando {filename}")

# Cargar con GeoPandas
print("\n🔄 Procesando archivo...")
municipios_gdf = gpd.read_file(filename)

print(f"\n✅ Archivo cargado exitosamente!")
print(f"   📊 Registros encontrados: {len(municipios_gdf)}")
print(f"   🗺️  Sistema de coordenadas: {municipios_gdf.crs}")

# Mostrar columnas disponibles
cols = municipios_gdf.columns.drop('geometry').tolist()
if cols:
    print(f"   📋 Columnas: {', '.join(cols[:5])}")
    if len(cols) > 5:
        print(f"              ... y {len(cols)-5} más")

# Reproyectar a WGS84 si es necesario
if municipios_gdf.crs != 'EPSG:4326':
    print(f"\n🔄 Reproyectando a WGS84 (EPSG:4326)...")
    municipios_gdf = municipios_gdf.to_crs('EPSG:4326')
    print(f"   ✅ Reproyección completada")

# Mostrar vista previa
print(f"\n📋 Vista previa de los datos:")
print("="*70)
display(municipios_gdf.head())
print("="*70)


In [1]:
#@title Copyright 2020 The Earth Engine Community Authors { display-mode: "form" }
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 🛰️ Generación de Mosaicos Sentinel-2 Libres de Nubes

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/famartinezal/sentinel1-change-detection/blob/master/05_mosaicos_sentinel2_sin_nubes.ipynb)

**[▶️ Abrir en Google Colab](https://colab.research.google.com/github/famartinezal/sentinel1-change-detection/blob/master/05_mosaicos_sentinel2_sin_nubes.ipynb)**

---

## 📋 ¿Qué hace este notebook?

Este notebook genera **mosaicos ópticos libres de nubes** a partir de imágenes Sentinel-2 para cualquier área de interés en Colombia (o el mundo).

### ✨ Características principales

- 🌐 **Ejecuta 100% en la nube** (Google Colab)
- 📁 **Sube tu propio polígono** (GPKG, Shapefile, GeoJSON)
- ☁️ **Elimina nubes automáticamente** con s2cloudless
- 📊 **Calcula índices espectrales** (NDVI, NDWI, EVI)
- 🗺️ **Visualización interactiva** con mapas
- 💾 **Descarga resultados** (CSV + GeoTIFF)

### 🚀 Inicio rápido

1. **Click en "▶️ Abrir en Google Colab"** arriba
2. Ejecuta las celdas en orden (Shift + Enter)
3. Sube tu archivo cuando se te pida
4. Ajusta las fechas según tu interés
5. ¡Espera los resultados!

**⏱️ Tiempo estimado**: 5-10 minutos (dependiendo del tamaño del área)

---

**Autor**: Adaptado del tutorial [s2cloudless](https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless) de jdbcode


---

## 📖 Tabla de Contenidos

1. [🔐 Autenticación Earth Engine](#autenticacion)
2. [📁 Subir Área de Interés](#subir-archivo)
3. [📅 Configurar Fechas y Parámetros](#configurar-parametros)
4. [☁️ Procesamiento y Enmascaramiento](#procesamiento)
5. [🗺️ Visualización de Resultados](#visualizacion)
6. [💾 Descarga de Datos](#descarga)

---

## ℹ️ Información Técnica

### Metodología de Enmascaramiento de Nubes

Este notebook utiliza el dataset **s2cloudless** que proporciona probabilidades de nubosidad pixel a pixel. El proceso incluye:

1. **Detección de nubes**: Usando probabilidades s2cloudless
2. **Detección de sombras**: Proyección direccional desde nubes
3. **Filtrado espacial**: Eliminación de parches pequeños
4. **Buffer de seguridad**: Dilatación de máscaras para mayor robustez

### Índices Espectrales Calculados

- **NDVI** (Normalized Difference Vegetation Index): Salud de la vegetación
- **NDWI** (Normalized Difference Water Index): Contenido de agua
- **EVI** (Enhanced Vegetation Index): Vegetación mejorado, menos sensible a saturación

### Requisitos del Archivo de Entrada

- **Formato**: GeoPackage (.gpkg), Shapefile (.shp + auxiliares), o GeoJSON (.geojson)
- **Geometría**: Polígono o MultiPolígono
- **CRS**: Cualquiera (se reproyecta automáticamente a WGS84)
- **Tamaño recomendado**: < 10,000 km² para procesamiento rápido


---

<a id="autenticacion"></a>
## 🔐 Paso 1: Autenticación Google Earth Engine

**⚠️ IMPORTANTE**: Ejecuta esta celda PRIMERO

Esta celda:
- Instala las dependencias necesarias (geemap)
- Autentica tu cuenta de Google Earth Engine
- Inicializa la conexión con Earth Engine

**📝 Nota**: La primera vez te pedirá que autorices el acceso a Earth Engine a través de tu cuenta de Google.


In [ ]:
# ============================================================================
# AUTENTICACIÓN E INICIALIZACIÓN DE GOOGLE EARTH ENGINE
# ============================================================================

# Detectar si estamos en Colab o entorno local
try:
    import google.colab
    IN_COLAB = True
    print("🌐 Ejecutando en Google Colab")
except:
    IN_COLAB = False
    print("💻 Ejecutando en entorno local")

# Instalar dependencias solo en Colab
if IN_COLAB:
    print("\n📦 Instalando dependencias...")
    !pip install -q geemap earthengine-api
    print("✓ Dependencias instaladas")

import ee

# Autenticar Earth Engine
print("\n🔐 Autenticando Google Earth Engine...")
print("   (Sigue las instrucciones si es la primera vez)")
ee.Authenticate()

# Inicializar Earth Engine
# IMPORTANTE: Reemplaza 'ee-famageoia' con tu propio project ID
ee.Initialize(project='ee-famageoia')

print("\n✅ Earth Engine inicializado correctamente")
print("   Listo para procesar datos satelitales!")


---

<a id="subir-archivo"></a>
## 📁 Paso 2: Subir tu Área de Interés (AOI)

**🎯 ¿Qué necesitas?**

Un archivo con el polígono de tu área de estudio en uno de estos formatos:
- `.gpkg` (GeoPackage) - **Recomendado**
- `.shp` + archivos auxiliares (Shapefile)
- `.geojson` (GeoJSON)

**💡 Consejos**:
- El área puede ser un municipio, finca, región, etc.
- Si tienes un Shapefile, sube **todos** los archivos (.shp, .shx, .dbf, .prj)
- El sistema reproyecta automáticamente a WGS84 si es necesario

**▶️ Ejecuta la siguiente celda** y selecciona tu archivo cuando se te pida.


In [ ]:
# ============================================================================
# CONVERTIR POLÍGONO(S) CARGADO(S) A AOI DE EARTH ENGINE
# ============================================================================

print("="*70)
print("ÁREA DE INTERÉS (AOI)")
print("="*70)

# El archivo cargado es el AOI completo
# Si tiene múltiples polígonos, se unifican en un solo AOI

print(f"  Número de polígonos: {len(municipios_gdf)}")

# Calcular área total y centroide
total_area_ha = municipios_gdf.geometry.area.sum() / 10000  # m² a hectáreas
aoi_unified = municipios_gdf.unary_union  # Unificar todos los polígonos
centroid = aoi_unified.centroid

print(f"  Área total: {total_area_ha:.2f} hectáreas ({total_area_ha/100:.2f} km²)")
print(f"  Centroide: ({centroid.y:.4f}, {centroid.x:.4f})")

# Convertir toda la geometría a Earth Engine
import json
geom_geojson = json.loads(municipios_gdf.to_json())

# Crear ee.Geometry desde todas las features
all_features = []
for feature in geom_geojson['features']:
    geom_coords = feature['geometry']
    if geom_coords['type'] == 'Polygon':
        all_features.append(ee.Geometry.Polygon(geom_coords['coordinates']))
    elif geom_coords['type'] == 'MultiPolygon':
        all_features.append(ee.Geometry.MultiPolygon(geom_coords['coordinates']))

# Si hay múltiples geometrías, unirlas
if len(all_features) == 1:
    AOI = all_features[0]
else:
    # Unir todos los polígonos en un solo MultiPolygon
    AOI = ee.Geometry.MultiPolygon([geom.coordinates().getInfo() for geom in all_features])

print(f"  Tipo de geometría EE: {AOI.type().getInfo()}")

# Nombre para el área (usar nombre de columna si existe, sino genérico)
if len(municipios_gdf) == 1:
    area_nombre = municipios_gdf.iloc[0].get('MPIO_CNMBR', 
                  municipios_gdf.iloc[0].get('NOMBRE',
                  municipios_gdf.iloc[0].get('nombre', 'Area_de_interes')))
else:
    area_nombre = f'AOI_{len(municipios_gdf)}_poligonos'

print(f"\n✓ AOI configurado: {area_nombre}")
print("="*70)

---

<a id="configurar-parametros"></a>
## 📅 Paso 3: Configurar Fechas y Parámetros

**🎯 Selecciona tu período de análisis de forma interactiva**

Usa los selectores de fecha para definir:
- 📆 **Fecha de inicio**: Primer día del período
- 📆 **Fecha de fin**: Último día del período

**⚙️ Parámetros opcionales**:
- **% Nubes máx**: Filtro inicial de imágenes (70% recomendado)
- **Umbral nube**: Sensibilidad de detección (40% recomendado)

**💡 Recomendaciones**:
- Períodos de 3-6 meses para mejor cobertura
- Evita épocas muy lluviosas si quieres menos nubes
- En Colombia: diciembre-marzo (menos nubes), abril-noviembre (más lluvia)

**👇 Ejecuta la celda y usa los widgets para seleccionar tus fechas**


In [ ]:
# ============================================================================
# SELECTOR INTERACTIVO DE FECHAS
# ============================================================================

# Importar widgets para selección interactiva
if IN_COLAB:
    from google.colab import widgets
    from datetime import date, datetime
    import ipywidgets as widgets_ipy
else:
    import ipywidgets as widgets_ipy
    from datetime import date, datetime

print("📅 SELECCIONA EL RANGO DE FECHAS PARA TU ANÁLISIS\n")

# Crear selectores de fecha
start_date_widget = widgets_ipy.DatePicker(
    description='Fecha Inicio:',
    value=date(2024, 7, 1),
    disabled=False
)

end_date_widget = widgets_ipy.DatePicker(
    description='Fecha Fin:',
    value=date(2024, 12, 31),
    disabled=False
)

# Parámetros avanzados con sliders
print("⚙️ Parámetros de enmascaramiento (valores recomendados):\n")

cloud_filter_widget = widgets_ipy.IntSlider(
    value=70,
    min=10,
    max=100,
    step=10,
    description='% Nubes máx:',
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

cld_prb_thresh_widget = widgets_ipy.IntSlider(
    value=40,
    min=10,
    max=90,
    step=10,
    description='Umbral nube:',
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

# Mostrar widgets
display(start_date_widget)
display(end_date_widget)
print("\n⚙️ Parámetros avanzados (opcional):")
display(cloud_filter_widget)
display(cld_prb_thresh_widget)

# Botón para confirmar
confirm_button = widgets_ipy.Button(
    description='✅ Confirmar Fechas',
    button_style='success',
    tooltip='Click para confirmar y continuar',
    icon='check'
)

output_area = widgets_ipy.Output()

def on_confirm_clicked(b):
    with output_area:
        output_area.clear_output()
        
        # Obtener valores seleccionados
        global START_DATE, END_DATE, CLOUD_FILTER, CLD_PRB_THRESH
        global NIR_DRK_THRESH, CLD_PRJ_DIST, BUFFER
        
        START_DATE = start_date_widget.value.strftime('%Y-%m-%d')
        END_DATE = end_date_widget.value.strftime('%Y-%m-%d')
        CLOUD_FILTER = cloud_filter_widget.value
        CLD_PRB_THRESH = cld_prb_thresh_widget.value
        
        # Parámetros fijos (pueden modificarse aquí si se desea)
        NIR_DRK_THRESH = 0.15
        CLD_PRJ_DIST = 2
        BUFFER = 100
        
        # Mostrar confirmación
        print("="*70)
        print("⚙️  CONFIGURACIÓN CONFIRMADA")
        print("="*70)
        print(f"\n📍 Área de Interés:")
        print(f"   Nombre: {area_nombre}")
        print(f"   Área: {total_area_ha:,.0f} hectáreas ({total_area_ha/100:,.0f} km²)")
        print(f"\n📅 Período de Análisis:")
        print(f"   Inicio: {START_DATE}")
        print(f"   Fin:    {END_DATE}")
        
        # Calcular días
        d1 = datetime.strptime(START_DATE, '%Y-%m-%d')
        d2 = datetime.strptime(END_DATE, '%Y-%m-%d')
        dias = (d2 - d1).days
        print(f"   Duración: {dias} días ({dias/30:.1f} meses)")
        
        print(f"\n☁️  Parámetros de Nubes:")
        print(f"   Filtro de nubes: {CLOUD_FILTER}%")
        print(f"   Umbral de probabilidad: {CLD_PRB_THRESH}%")
        print(f"   Buffer de seguridad: {BUFFER} m")
        print("\n" + "="*70)
        print("✅ Configuración lista. Ejecuta la siguiente celda para continuar.")
        print("="*70)

confirm_button.on_click(on_confirm_clicked)

print("\n")
display(confirm_button)
display(output_area)

print("\n💡 Consejos:")
print("   • Períodos de 3-6 meses funcionan mejor")
print("   • Para Colombia, diciembre-marzo tiene menos nubes")
print("   • Mayor % nubes = más imágenes disponibles")


---

<a id="procesamiento"></a>
## ☁️ Paso 4: Procesamiento Automático

**🤖 Las siguientes celdas procesan automáticamente tus datos**

El notebook realizará:
1. ✅ Construcción de la colección Sentinel-2
2. ✅ Detección y enmascaramiento de nubes
3. ✅ Detección y enmascaramiento de sombras
4. ✅ Generación del mosaico compuesto (mediana)
5. ✅ Cálculo de índices espectrales (NDVI, NDWI, EVI)

**⏱️ Tiempo estimado**: 2-5 minutos

**💡 Tip**: Puedes ejecutar todas las celdas restantes de una vez:
- En Colab: Menú → Runtime → Run after
- Con teclado: Shift + Enter en cada celda

---

### 🔧 Funciones de Procesamiento


In [ ]:
# ============================================================================
# FUNCIÓN PARA CONSTRUIR COLECCIÓN SENTINEL-2 CON S2CLOUDLESS
# ============================================================================

def get_s2_sr_cld_col(aoi, start_date, end_date, cloud_filter):
    """
    Construye una colección Sentinel-2 SR con datos de probabilidad de nubes.
    
    Args:
        aoi: ee.Geometry - Área de interés
        start_date: str - Fecha inicio 'YYYY-MM-DD'
        end_date: str - Fecha fin 'YYYY-MM-DD'
        cloud_filter: int - % máximo de cobertura de nubes
    
    Returns:
        ee.ImageCollection - Colección SR con propiedad 's2cloudless'
    """
    # Importar y filtrar Sentinel-2 SR
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', cloud_filter)))

    # Importar y filtrar s2cloudless
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Unir ambas colecciones por 'system:index'
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

print("✓ Función get_s2_sr_cld_col() definida")


In [ ]:
# Construir la colección filtrada por el municipio y fechas
print("\n🔍 Construyendo colección Sentinel-2...")
s2_sr_cld_col = get_s2_sr_cld_col(AOI, START_DATE, END_DATE, CLOUD_FILTER)

# Obtener información de la colección
n_images = s2_sr_cld_col.size().getInfo()
print(f"✓ Colección construida: {n_images} imágenes encontradas")

if n_images == 0:
    print("\n⚠️  ADVERTENCIA: No se encontraron imágenes con los criterios especificados.")
    print("    Intente:")
    print("    - Ampliar el rango de fechas")
    print("    - Aumentar CLOUD_FILTER (permitir más nubes)")
    print("    - Verificar que el municipio tenga cobertura Sentinel-2")
else:
    # Mostrar fechas de imágenes disponibles
    dates_list = s2_sr_cld_col.aggregate_array('system:time_start').getInfo()
    import datetime
    dates = [datetime.datetime.fromtimestamp(d/1000).strftime('%Y-%m-%d') for d in dates_list[:10]]
    print(f"\n📅 Primeras fechas disponibles: {', '.join(dates)}")
    if n_images > 10:
        print(f"    ... y {n_images - 10} más")


In [ ]:
# ============================================================================
# FUNCIONES DE ENMASCARAMIENTO
# ============================================================================

def add_cloud_bands(img):
    """Agrega bandas de probabilidad de nube y máscara binaria de nubes."""
    # Obtener probabilidad de nube desde s2cloudless
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')
    
    # Generar máscara binaria basada en umbral
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')
    
    # Agregar bandas a la imagen
    return img.addBands(ee.Image([cld_prb, is_cloud]))


def add_shadow_bands(img):
    """Agrega bandas de detección de sombras de nubes."""
    # Identificar píxeles de agua (para excluirlos de sombras)
    not_water = img.select('SCL').neq(6)
    
    # Identificar píxeles oscuros en NIR (sombras potenciales)
    SR_BAND_SCALE = 1e4
    dark_pixels = (img.select('B8')
                   .lt(NIR_DRK_THRESH * SR_BAND_SCALE)
                   .multiply(not_water)
                   .rename('dark_pixels'))
    
    # Calcular azimut de proyección de sombras
    shadow_azimuth = ee.Number(90).subtract(
        ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE'))
    )
    
    # Proyectar sombras desde nubes
    cld_proj = (img.select('clouds')
                .directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST * 10)
                .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
                .select('distance')
                .mask()
                .rename('cloud_transform'))
    
    # Intersección de píxeles oscuros con proyección de sombras
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')
    
    # Agregar bandas de sombras
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))


def add_cld_shdw_mask(img):
    """Genera la máscara final combinando nubes y sombras."""
    # Agregar componentes de nubes
    img_cloud = add_cloud_bands(img)
    
    # Agregar componentes de sombras
    img_cloud_shadow = add_shadow_bands(img_cloud)
    
    # Combinar nubes y sombras (valor 1 = nube/sombra, 0 = despejado)
    is_cld_shdw = (img_cloud_shadow.select('clouds')
                   .add(img_cloud_shadow.select('shadows'))
                   .gt(0))
    
    # Eliminar parches pequeños y dilatar con BUFFER
    is_cld_shdw = (is_cld_shdw
                   .focalMin(2)
                   .focalMax(BUFFER * 2 / 20)
                   .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
                   .rename('cloudmask'))
    
    # Agregar máscara final
    return img_cloud_shadow.addBands(is_cld_shdw)


def apply_cld_shdw_mask(img):
    """Aplica la máscara de nubes/sombras a las bandas de reflectancia."""
    # Invertir máscara (0 = nube/sombra, 1 = despejado)
    not_cld_shdw = img.select('cloudmask').Not()
    
    # Aplicar máscara solo a bandas de reflectancia (B.*)
    return img.select('B.*').updateMask(not_cld_shdw)


print("✓ Funciones de enmascaramiento definidas:")
print("  - add_cloud_bands()")
print("  - add_shadow_bands()")
print("  - add_cld_shdw_mask()")
print("  - apply_cld_shdw_mask()")


In [ ]:
# ============================================================================
# GENERAR MOSAICO COMPUESTO (MEDIANA) LIBRE DE NUBES
# ============================================================================

print("\n🎨 Generando mosaico libre de nubes...")

# Aplicar máscaras a la colección y reducir por mediana
s2_sr_median = (s2_sr_cld_col
                .map(add_cld_shdw_mask)
                .map(apply_cld_shdw_mask)
                .median()
                .clip(AOI))  # Recortar al límite del municipio

print("✓ Mosaico generado exitosamente")
print(f"  Método de composición: Mediana")
print(f"  Bandas disponibles: {s2_sr_median.bandNames().getInfo()}")

# Calcular índices espectrales
print("\n📊 Calculando índices espectrales...")

# NDVI = (NIR - Red) / (NIR + Red)
ndvi = s2_sr_median.normalizedDifference(['B8', 'B4']).rename('NDVI')

# NDWI = (Green - NIR) / (Green + NIR) 
ndwi = s2_sr_median.normalizedDifference(['B3', 'B8']).rename('NDWI')

# EVI = 2.5 * ((NIR - Red) / (NIR + 6*Red - 7.5*Blue + 1))
evi = s2_sr_median.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
        'NIR': s2_sr_median.select('B8'),
        'RED': s2_sr_median.select('B4'),
        'BLUE': s2_sr_median.select('B2')
    }).rename('EVI')

# Agregar índices al mosaico
s2_sr_median = s2_sr_median.addBands([ndvi, ndwi, evi])

print("✓ Índices calculados:")
print("  - NDVI (vegetación)")
print("  - NDWI (agua/humedad)")
print("  - EVI (vegetación mejorado)")


---

<a id="visualizacion"></a>
## 🗺️ Paso 5: Visualización Interactiva

**📊 Exploración visual de resultados**

Vamos a crear un mapa interactivo con **geemap** que incluye:
- 🌍 Composición RGB (color verdadero)
- 🌲 Composición falso color (NIR-R-G)
- 🌿 NDVI (vegetación)
- 💧 NDWI (agua/humedad)
- 📈 EVI (vegetación mejorado)

**💡 Controles del mapa**:
- Usa el panel de capas para activar/desactivar visualizaciones
- Zoom con scroll o botones +/-
- Mide distancias y áreas con las herramientas


In [ ]:
# ============================================================================
# CREAR MAPA INTERACTIVO CON GEEMAP
# ============================================================================

import geemap

print("🗺️  Creando mapa interactivo con geemap...\n")

# Crear mapa centrado en el AOI
Map = geemap.Map()

# Centrar el mapa en el AOI
Map.centerObject(AOI, zoom=11)

# CAPAS DE VISUALIZACIÓN

# 1. Composición RGB (Color verdadero)
rgb_vis = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000,
    'gamma': 1.2
}
Map.addLayer(s2_sr_median, rgb_vis, '🌍 RGB (Color verdadero)', True)

# 2. Composición Falso Color (NIR-Red-Green) - Ideal para vegetación
false_color_vis = {
    'bands': ['B8', 'B4', 'B3'],
    'min': 0,
    'max': 3500,
    'gamma': 1.1
}
Map.addLayer(s2_sr_median, false_color_vis, '🌲 Falso color (NIR-R-G)', False)

# 3. NDVI (Índice de vegetación)
ndvi_vis = {
    'min': -0.2,
    'max': 0.9,
    'palette': ['brown', 'yellow', 'lightgreen', 'green', 'darkgreen']
}
Map.addLayer(s2_sr_median.select('NDVI'), ndvi_vis, '🌿 NDVI', False)

# 4. NDWI (Índice de agua/humedad)
ndwi_vis = {
    'min': -0.3,
    'max': 0.5,
    'palette': ['red', 'yellow', 'white', 'cyan', 'blue']
}
Map.addLayer(s2_sr_median.select('NDWI'), ndwi_vis, '💧 NDWI', False)

# 5. EVI (Vegetación mejorado)
evi_vis = {
    'min': -0.2,
    'max': 1.0,
    'palette': ['white', 'lightgreen', 'green', 'darkgreen']
}
Map.addLayer(s2_sr_median.select('EVI'), evi_vis, '📈 EVI', False)

# Agregar contorno del AOI
empty = ee.Image().byte()
outline = empty.paint(
    featureCollection=ee.FeatureCollection([ee.Feature(AOI)]),
    color=1,
    width=3
)
Map.addLayer(outline, {'palette': 'red'}, '📍 Límite del AOI', True)

# Agregar leyendas para los índices
print("✅ Capas agregadas al mapa")
print("\n📋 Interpretación de índices:")
print("\n🌿 NDVI (Normalized Difference Vegetation Index):")
print("   < 0.2  : Suelo desnudo, rocas, agua")
print("   0.2-0.4: Vegetación dispersa")
print("   0.4-0.6: Vegetación moderada")
print("   > 0.6  : Vegetación densa y saludable")
print("\n💧 NDWI (Normalized Difference Water Index):")
print("   < 0    : Suelo seco, vegetación")
print("   0-0.2  : Vegetación con humedad")
print("   > 0.2  : Agua, humedales")
print("\n📈 EVI (Enhanced Vegetation Index):")
print("   Similar al NDVI pero menos sensible a saturación")
print("   Mejor para áreas de vegetación muy densa")

print("\n" + "="*70)
print("🎯 Usa el panel de capas para explorar diferentes visualizaciones")
print("="*70)

# Mostrar el mapa
Map


---

<a id="descarga"></a>
## 💾 Paso 6: Descarga de Resultados

**📊 Obtén tus datos procesados**

Tienes dos opciones:
1. **CSV con estadísticas** (descarga inmediata)
2. **GeoTIFF con todas las bandas** (exporta a Google Drive)

---

### 📈 Estadísticas del AOI


In [ ]:
# ============================================================================
# CALCULAR ESTADÍSTICAS DEL MOSAICO
# ============================================================================

print("\n📊 Calculando estadísticas del mosaico...\n")

# Calcular estadísticas para cada índice
indices = ['NDVI', 'NDWI', 'EVI']
stats = {}

for idx_name in indices:
    print(f"Calculando {idx_name}...")
    idx_stats = s2_sr_median.select(idx_name).reduceRegion(
        reducer=ee.Reducer.mean().combine(
            reducer2=ee.Reducer.stdDev(),
            sharedInputs=True
        ).combine(
            reducer2=ee.Reducer.minMax(),
            sharedInputs=True
        ),
        geometry=AOI,
        scale=10,
        maxPixels=1e9
    ).getInfo()
    
    stats[idx_name] = {
        'mean': idx_stats.get(f'{idx_name}_mean'),
        'stdDev': idx_stats.get(f'{idx_name}_stdDev'),
        'min': idx_stats.get(f'{idx_name}_min'),
        'max': idx_stats.get(f'{idx_name}_max')
    }

print("\n" + "="*70)
print(f"ESTADÍSTICAS - {area_nombre} ({START_DATE} a {END_DATE})")
print("="*70)

for idx_name, values in stats.items():
    if values['mean'] is not None:
        print(f"\n{idx_name}:")
        print(f"  Media:   {values['mean']:.4f}")
        print(f"  Std Dev: {values['stdDev']:.4f}")
        print(f"  Mínimo:  {values['min']:.4f}")
        print(f"  Máximo:  {values['max']:.4f}")
    else:
        print(f"\n{idx_name}: No hay datos disponibles")

print("\n" + "="*70)

# Crear DataFrame con resultados
import pandas as pd

results_df = pd.DataFrame({
    'Area': [area_nombre],
    'Fecha_inicio': [START_DATE],
    'Fecha_fin': [END_DATE],
    'N_imagenes': [n_images],
    'Area_ha': [total_area_ha],
    'NDVI_mean': [stats['NDVI']['mean']],
    'NDVI_std': [stats['NDVI']['stdDev']],
    'NDWI_mean': [stats['NDWI']['mean']],
    'NDWI_std': [stats['NDWI']['stdDev']],
    'EVI_mean': [stats['EVI']['mean']],
    'EVI_std': [stats['EVI']['stdDev']]
})

print("\n✓ Estadísticas calculadas")


In [ ]:
# ============================================================================
# DESCARGAR ESTADÍSTICAS (CSV)
# ============================================================================

print("\n💾 Preparando descarga de estadísticas...")

# Guardar CSV
csv_filename = f'estadisticas_{area_nombre.replace(" ", "_")}_{START_DATE}_{END_DATE}.csv'
results_df.to_csv(csv_filename, index=False)

print(f"✓ Archivo CSV creado: {csv_filename}")

# Descargar en Colab
if IN_COLAB:
    from google.colab import files
    files.download(csv_filename)
    print("✓ Descarga iniciada en tu navegador")
else:
    print(f"✓ Archivo guardado localmente: {csv_filename}")

# Mostrar tabla
print("\n📋 Resumen de resultados:")
print(results_df.to_string(index=False))


---

### 🌍 Exportar Imagen a Google Drive (Opcional)

**📦 Exporta el mosaico completo como GeoTIFF**

Esta celda inicia una tarea de exportación en segundo plano. El archivo se guardará en tu Google Drive cuando termine (puede tomar varios minutos dependiendo del tamaño del área).

**🔍 Incluye**:
- Bandas espectrales: B2, B3, B4, B8, B11, B12
- Índices: NDVI, NDWI, EVI
- Resolución: 10 metros
- Formato: GeoTIFF (compatible con QGIS, ArcGIS, etc.)

**💡 Monitorea el progreso**: https://code.earthengine.google.com/tasks


In [ ]:
# ============================================================================
# EXPORTAR MOSAICO A GOOGLE DRIVE
# ============================================================================

print("🚀 Iniciando exportación a Google Drive...\n")

# Seleccionar bandas a exportar
bands_to_export = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12', 'NDVI', 'NDWI', 'EVI']
image_to_export = s2_sr_median.select(bands_to_export)

# Nombre del archivo
export_name = f'S2_Mosaic_{area_nombre.replace(" ", "_")}_{START_DATE}_{END_DATE}'

# Configurar y lanzar tarea de exportación
task = ee.batch.Export.image.toDrive(
    image=image_to_export,
    description=export_name,
    folder='EarthEngine_Exports',
    fileNamePrefix=export_name,
    scale=10,  # Resolución: 10 metros
    region=AOI,
    maxPixels=1e9,
    crs='EPSG:4326',
    fileFormat='GeoTIFF'
)

task.start()

# Mostrar información de la exportación
print("="*70)
print("✅ TAREA DE EXPORTACIÓN INICIADA")
print("="*70)
print(f"\n📦 Detalles de la exportación:")
print(f"   Nombre:      {export_name}")
print(f"   Carpeta:     EarthEngine_Exports (Google Drive)")
print(f"   Bandas:      {', '.join(bands_to_export)}")
print(f"   Resolución:  10 metros")
print(f"   Área:        {total_area_ha:,.0f} ha ({total_area_ha/100:,.0f} km²)")
print(f"   Formato:     GeoTIFF")
print("\n" + "="*70)
print("⏳ Procesamiento en segundo plano iniciado")
print("="*70)
print("\n📊 Monitorea el progreso en:")
print("   https://code.earthengine.google.com/tasks")
print("\n💾 El archivo aparecerá en Google Drive cuando termine")
print("   (puede tomar varios minutos dependiendo del área)")
print("\n💡 Tip: Puedes cerrar este notebook, la tarea seguirá corriendo")
